In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import time

In [2]:
target = pd.read_csv('targets.csv')
structure = pd.read_csv('structure.csv')
user_activity = pd.read_csv('user_activity.csv')
user_activity_test = pd.read_csv('user_activity_test.csv')

## Target

In [3]:
try: 
    target.drop('distinction', axis = 1, inplace = True)
except Exception: 
    pass

target.head()

,user_id,score,passed
0,1,0,0
1,2,82,0
2,3,89,0
3,4,9,0
4,6,26,0


## Structure of Steps

In [4]:
try:
    structure = structure.sort_values(['module_position', 'lesson_position', 'step_position'])
    structure['step_num'] = range(0, len(structure))
except Exception:
    pass

structure = structure[['step_id','step_num', 'step_type', 'step_cost']]

structure.head()

,step_id,step_num,step_type,step_cost
0,7225,0,text,0
1,7230,1,text,0
2,7231,2,text,0
3,7232,3,text,0
4,43604,4,text,0


### Функиця до добавлению доп. колонок в "рабочую" таблицу

In [5]:
def add_columns(df):
    
    df['time_f'] = pd.to_datetime(df['time'], unit='s')
    
    df = pd.merge(
    left = df,
    right = structure[['step_id', 'step_num']], 
    how = 'left', 
    on = 'step_id')
    
    df = df[['user_id', 'action', 'step_num', 'step_type', 'step_cost', 'time', 'time_f']]
    
    return df

In [6]:
try:
    user_activity = add_columns(user_activity)
except Exception:
    pass
user_activity.head()

,user_id,action,step_num,step_type,step_cost,time,time_f
0,5920,discovered,81,text,0,1403165936,2014-06-19 08:18:56
1,5920,passed,81,text,0,1403165936,2014-06-19 08:18:56
2,5920,viewed,81,text,0,1403165936,2014-06-19 08:18:56
3,5920,passed,93,text,0,1403165946,2014-06-19 08:19:06
4,5920,viewed,93,text,0,1403165946,2014-06-19 08:19:06


### Функция подсчёта баллов за пройденные степы по юзерам, а также кол-во пройденных степов

In [7]:
def user_score(df):
    
    users = pd.DataFrame(df['user_id'].unique(), columns=['user_id'])
    df = df[df['action'] == 'passed']
    
    df['step_cost_cat'] = df['step_cost'].astype('str')
    pivot = pd.pivot_table(df,index=["user_id"],values=["step_cost"], columns = 'step_cost_cat', aggfunc='sum')
    pivot.reset_index(level=0, inplace=True)
    
    columns_names = ["cost_"+str(i) for i in sorted(df["step_cost"].unique().tolist())]
    pivot.columns = ['user_id'] + columns_names
    
    pivot.fillna(0, inplace = True)
    
    pivot['sum_of_step_cost'] = pivot['cost_1'] + pivot['cost_2'] + pivot['cost_3'] + pivot['cost_4']
    pivot['count_of_steps_passed'] = pivot['cost_1'] + pivot['cost_2']/2 + pivot['cost_3']/3 + pivot['cost_4']/4
    pivot.drop('cost_0', axis=1, inplace = True)
    pivot = pd.merge(left = pivot, right = users, how = 'outer', on = 'user_id')
    pivot.fillna(0, inplace = True)
    
    return pivot

In [8]:
user_score(user_activity).head()

C:\Users\Adamo\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,user_id,cost_1,cost_2,cost_3,cost_4,sum_of_step_cost,count_of_steps_passed
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,9.0,0.0,0.0,0.0,9.0,9.0
3,4.0,9.0,0.0,0.0,0.0,9.0,9.0
4,6.0,14.0,12.0,0.0,0.0,26.0,20.0


### Функция подсчёта количества просмотренны, но непройденных степов, а также сумма "упущенных" баллов, за просмотренные, но не сделанные степы.

In [9]:
def missed_steps_and_score(df):
    #df_filter = (df['action'] == 'passed') & (df['step_cost'] != 0)
    df_filter = (df['action'] == 'passed')
    pivot1 = pd.DataFrame(df[df_filter].groupby(['user_id', 'step_num'], as_index = False)['step_cost'].mean())
    
    #df_filter = (df['action'] != 'passed') & (df['step_cost'] != 0)
    df_filter = (df['action'] != 'passed')
    pivot2 = pd.DataFrame(df[df_filter].groupby(['user_id', 'step_num'], as_index = False)['step_cost'].mean())
    
    pivot = pd.merge(
        left = pivot1,
        right = pivot2,
        how = 'outer',
        on = ['user_id', 'step_num'])
    
    pivot.fillna(0, inplace = True)
    
    pivot['missing_step_cost'] = (pivot['step_cost_y'] - pivot['step_cost_x'])
    pivot = pivot[pivot['missing_step_cost'] != 0]
    pivot = pd.DataFrame(pivot.groupby(['user_id'], as_index = False)['missing_step_cost'].agg(['sum', 'count']))
    pivot.reset_index(drop = False, inplace = True)
    pivot.columns = ['user_id', 'missed_score', 'missed_steps_N']        
    
    return pivot    

In [10]:
missed_steps_and_score(user_activity).head()

,user_id,missed_score,missed_steps_N
0,1.0,109.0,68
1,7.0,16.0,8
2,12.0,1.0,1
3,14.0,2.0,1
4,15.0,2.0,1


### Функция рассчёта темпа прохождения степов пользователем (используется на первых 50 степах)

In [11]:
def user_passing_rate(df):
    
    # найдем для каждого пользователя и проеденного степа время прохождения и время первой попытки решить степ.
    # добавим его в 'passing_time'
    
    df = df[df['step_num'] < 50]
    time_passed = df[(df['action'] == 'passed') & (df['step_cost'] != 0)].groupby(['user_id', 'step_num'], as_index=False)['time'].max()
    time_started_attempt = df[(df['action'] == 'started_attempt') & (df['step_cost'] != 0)].groupby(['user_id', 'step_num'], as_index=False)['time'].min()

    df = pd.merge(
        left = time_passed,
        right = time_started_attempt, 
        how = 'left', 
        on = ['user_id', 'step_num'])
    
    df.columns = ['user_id', 'step_num', 'time_pass', 'time_start']
    df['passing_time'] =  df['time_pass'] - df['time_start']
    
    # найдём минимальное и максимальное время прохождение стэпа
    
    min_step_passing_time = df.groupby(['step_num'], as_index=False)['passing_time'].min()
    max_step_passing_time = df.groupby(['step_num'], as_index=False)['passing_time'].max()
    max_time = 30 * 60
    max_step_passing_time[max_step_passing_time['passing_time'] > max_time]['passing_time'] = max_time
    
    df = pd.merge(
        left = df,
        right = min_step_passing_time, 
        how = 'left', 
        on = ['step_num'])
    
    df = pd.merge(
        left = df,
        right = max_step_passing_time, 
        how = 'left', 
        on = ['step_num'])
    
    df.columns = ['user_id', 'step_num', 'time_pass', 'time_start', 'passing_time', 'min_passing_time', 'max_passing_time']
    
    # пролагарифмируем затраченное на прохождение время
    
    df['passing_time'] = np.log(df['passing_time'])
    df['min_passing_time'] = np.log(df['min_passing_time'])
    df['max_passing_time'] = np.log(df['max_passing_time'])
    
    # нормализуем затраченное на прохождение степа время
    
    df['norm_passing_time_rate'] = (df['passing_time'] - df['min_passing_time']) / (df['max_passing_time'] - df['min_passing_time'])
    
    # посчитаем как быстро в среднем пользователь проходил степы
    
    mean_user_passing_rate = df.groupby(['user_id'], as_index=False)['norm_passing_time_rate'].mean()
    
    return mean_user_passing_rate

In [12]:
user_passing_rate(user_activity).head()

,user_id,norm_passing_time_rate
0,3,0.251854
1,4,0.295214
2,6,0.178440
3,7,0.278024
4,8,0.201956


### Функция подсчёта активного периода у пользователя, а также количества активных дней

In [13]:
def user_daily_activity(df):
    
    # количество уникальных дней-занятий
    one_day = 24 * 60 * 60
    df['days'] = (df['time'] / one_day).astype(int)
    number_of_days = pd.DataFrame(df.groupby('user_id')['days'].nunique())
    
    # период активности, в днях
    user_first_last_day = df.groupby(['user_id'])['days'].agg([np.max, np.min])
    user_first_last_day['dayly_period'] = user_first_last_day.iloc[:, 0] - user_first_last_day.iloc[:, 1] + 1
    
    user_dayly_activity = pd.merge(
        left = user_first_last_day,
        right = number_of_days, 
        how = 'left',
        left_index = True,
        right_index = True)
    
    user_dayly_activity = user_dayly_activity[['dayly_period', 'days']]
    user_dayly_activity.columns = ['dayly_period', 'N_of_un_days']
    user_dayly_activity.reset_index(level=0, inplace=True)
    user_dayly_activity['regularity'] = user_dayly_activity['N_of_un_days'] / user_dayly_activity['dayly_period']
    
    return user_dayly_activity

In [14]:
user_daily_activity(user_activity).head()

,user_id,dayly_period,N_of_un_days,regularity
0,1,7,7,1.00
1,2,1,1,1.00
2,3,6,3,0.50
3,4,6,6,1.00
4,6,4,3,0.75


### Обработка данных с "user_activity"

In [15]:
def process_table(df):
    
    try:
        df = add_columns(df)
    except Exception:
        pass

    all_fun = [user_score, missed_steps_and_score, user_passing_rate, user_daily_activity]
    
    new_df = pd.DataFrame({'user_id' : []})
    
    for i in all_fun:
        new_df = pd.merge(
            left = new_df,
            right = i(df), 
            how = 'outer',
            on = 'user_id')
        
    new_df['norm_passing_time_rate'].fillna(1, inplace=True)
    new_df['missed_score'].fillna(0, inplace=True)
    new_df['missed_steps_N'].fillna(0, inplace=True)
    
    new_df['score_without_missed_score'] = new_df['sum_of_step_cost'] - new_df['missed_score']
    
    new_df = new_df[['user_id', 'cost_1', 'cost_2', 'cost_3', 'cost_4', 'sum_of_step_cost', 'missed_score', 'score_without_missed_score', 
          'count_of_steps_passed', 'missed_steps_N', 'norm_passing_time_rate', 
           'dayly_period', 'N_of_un_days', 'regularity']]

    return new_df


In [16]:
process_table(user_activity).head()

C:\Users\Adamo\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,user_id,cost_1,cost_2,cost_3,cost_4,sum_of_step_cost,missed_score,score_without_missed_score,count_of_steps_passed,missed_steps_N,norm_passing_time_rate,dayly_period,N_of_un_days,regularity
0,1.0,0.0,0.0,0.0,0.0,0.0,109.0,-109.0,0.0,68.0,1.000000,7,7,1.00
1,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1,1,1.00
2,3.0,9.0,0.0,0.0,0.0,9.0,0.0,9.0,9.0,0.0,0.251854,6,3,0.50
3,4.0,9.0,0.0,0.0,0.0,9.0,0.0,9.0,9.0,0.0,0.295214,6,6,1.00
4,6.0,14.0,12.0,0.0,0.0,26.0,0.0,26.0,20.0,0.0,0.178440,4,3,0.75


In [17]:
process_table(user_activity).info()

C:\Users\Adamo\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


<class 'pandas.core.frame.DataFrame'>
Int64Index: 16625 entries, 0 to 16624
Data columns (total 14 columns):
user_id                       16625 non-null float64
cost_1                        16625 non-null float64
cost_2                        16625 non-null float64
cost_3                        16625 non-null float64
cost_4                        16625 non-null float64
sum_of_step_cost              16625 non-null float64
missed_score                  16625 non-null float64
score_without_missed_score    16625 non-null float64
count_of_steps_passed         16625 non-null float64
missed_steps_N                16625 non-null float64
norm_passing_time_rate        16625 non-null float64
dayly_period                  16625 non-null int32
N_of_un_days                  16625 non-null int64
regularity                    16625 non-null float64
dtypes: float64(12), int32(1), int64(1)
memory usage: 1.8 MB


### Построение модели (линейной регрессии)

In [18]:
train = process_table(user_activity)
train = pd.merge(
    left = train,
    right = target, 
    how = 'left', 
    on = 'user_id')
train.head()

C:\Users\Adamo\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,user_id,cost_1,cost_2,cost_3,cost_4,sum_of_step_cost,missed_score,score_without_missed_score,count_of_steps_passed,missed_steps_N,norm_passing_time_rate,dayly_period,N_of_un_days,regularity,score,passed
0,1.0,0.0,0.0,0.0,0.0,0.0,109.0,-109.0,0.0,68.0,1.000000,7,7,1.00,0,0
1,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1,1,1.00,82,0
2,3.0,9.0,0.0,0.0,0.0,9.0,0.0,9.0,9.0,0.0,0.251854,6,3,0.50,89,0
3,4.0,9.0,0.0,0.0,0.0,9.0,0.0,9.0,9.0,0.0,0.295214,6,6,1.00,9,0
4,6.0,14.0,12.0,0.0,0.0,26.0,0.0,26.0,20.0,0.0,0.178440,4,3,0.75,26,0


#### Увеличим размер трэйна для того, чтобы уровнять кол-во "0" и "1" в "passed"

In [19]:
train_fit = train
for i in range(0, 24):
    train_fit = train_fit.append(train[train['passed'] == 1])
    i += 1

In [36]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

# модель с лучшими оценками, но которая не были опробована на кэгле
# features = ['count_of_steps_passed', 'missed_steps_N', 'norm_passing_time_rate', 'dayly_period']

# модель со всеми факторами, которая была опробована на кэгле
features = ['sum_of_step_cost', 'missed_score', 'score_without_missed_score','count_of_steps_passed', 'missed_steps_N', 'norm_passing_time_rate', 'dayly_period', 'N_of_un_days', 'regularity']

target = 'score'

model = LinearRegression()
model.fit(train_fit[features], train_fit[target])

train['prediction_score'] = model.predict(train[features])
train['predictions'] = 0
train.loc[train['prediction_score'] > 90, 'predictions'] = 1

In [37]:
print(f1_score(train['passed'], train['predictions']))
print(recall_score(train['passed'], train['predictions']))
print(accuracy_score(train['passed'], train['predictions']))

0.430953969313
0.490136570561
0.948691729323


### Random Forest, который не заработал

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.cross_validation import cross_val_predict
# from sklearn.cross_validation import KFold

# features = ['sum_of_step_cost']
# target = 'passed'

# rf = RandomForestClassifier(n_estimators = 100, max_depth=3, class_weight="balanced", random_state=1)
# kf = KFold(len(train), n_folds=5, random_state=1)
# predictions = cross_val_predict(rf, train[features], train[target], cv=kf)
# train['predictions'] = pd.Series(predictions)

### Предсказание теста по модели линейной регрессии

In [91]:
test = process_table(user_activity_test)

test['prediction_score'] = model.predict(test[features])
test['predictions'] = 0
test.loc[test['prediction_score'] > 90, 'predictions'] = 1

test['predictions'].value_counts()

C:\Users\Adamo\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0    4042
1     213
Name: predictions, dtype: int64

In [93]:
file = test[['user_id', 'predictions']]
file.columns = ['user_id', 'passed']
file['user_id'] = file['user_id'].astype('int')
file['passed'] = file['passed'].astype('int')
file.set_index('user_id', inplace = True)
file.to_csv('answers.csv')

C:\Users\Adamo\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\Adamo\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
